In [41]:
# 사용할 기본 패키지 로드
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import patches
%matplotlib inline
import os
import numpy as np

In [42]:
Basic_dir = os.path.dirname(os.path.dirname(os.path.realpath('EDA.ipynb')))
PATH = Basic_dir + '\\Data\\Data(HeartD_B)'
os.listdir(PATH)

['sample_submission(DT).csv',
 'sample_submission(KNN).csv',
 'sample_submission(LR).csv',
 'sample_submission(LR_6Factor).csv',
 'sample_submission(LR_CH).csv',
 'sample_submission(LR_CH1).csv',
 'sample_submission(LR_CH2).csv',
 'sample_submission(LR_CH3).csv',
 'sample_submission(LR_CH4).csv',
 'sample_submission(LR_PCA7).csv',
 'sample_submission(LR_slope&ageET).csv',
 'sample_submission(LR_slopeET).csv',
 'sample_submission(NB).csv',
 'sample_submission(NT).csv',
 'sample_submission(SVM).csv',
 'sample_submission(SVM_CH).csv',
 'sample_submission(SVM_CH1).csv',
 'sample_submission(SVM_CH2).csv',
 'sample_submission(SVM_slope&ageET).csv',
 'sample_submission(SVM_slopeET).csv',
 'sample_submission.csv',
 'test.csv',
 'train.csv']

In [43]:
# 결측값 없음 / 15개의 열과 151개의 행으로 이루어진 데이터 구조
df_train = pd.read_csv(PATH + '\\train.csv')
df_test = pd.read_csv(PATH + '\\test.csv')
df_train['age_gr'] = np.where(df_train.age < 20, 10, 
                     np.where(df_train.age < 30, 20, 
                     np.where(df_train.age < 40, 30, 
                     np.where(df_train.age < 50, 40,
                     np.where(df_train.age < 60, 50,
                     np.where(df_train.age < 70, 60, 70))))))
df_test['age_gr'] = np.where(df_test.age < 20, 10, 
                     np.where(df_test.age < 30, 20, 
                     np.where(df_test.age < 40, 30, 
                     np.where(df_test.age < 50, 40,
                     np.where(df_test.age < 60, 50,
                     np.where(df_test.age < 70, 60, 70))))))
df_train.drop(['id', 'age'], axis = 1, inplace = True)
df_test.drop(['id', 'age'], axis = 1, inplace = True)
df_train.head()

,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,age_gr
0,1,2,130,197,1,0,152,0,1.2,0,0,2,1,50
1,1,3,152,298,1,1,178,0,1.2,1,0,3,1,50
2,1,1,192,283,0,0,195,0,0.0,2,1,3,0,50
3,0,0,138,236,0,0,152,1,0.2,1,0,2,1,40
4,1,1,122,192,0,1,174,0,0.0,2,0,2,1,30


In [44]:
# ca가 영향력이 제일 큼
# ca에서 null 값은 1으로 변환 후 분석 40, 45, 79, 80, 95 # 어떤것으로 변환하느냐에 따라서 예측결과가 달라짐
# df_test[df_test.ca == 4] = df_test[df_test.ca == 4].apply(lambda x: 2)
# df_test.ca[95] 
df_test.ca.value_counts()

0    80
1    34
2    23
3    10
4     5
Name: ca, dtype: int64

In [45]:
x_list = ['thalach', 'oldpeak', 'slope', 'ca', 'sex', 'cp', 'exang', 'thal', 'age_gr']
transform_data1 = df_train[x_list]
transform_data2 = df_test[x_list] 

# 가장 일반적인 StandardScaler를 사용해서 정규화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler 
# scaler = MinMaxScaler()
scaler = StandardScaler()

std_transform_data1 = scaler.fit_transform(transform_data1)
std_transform_data2 = scaler.fit_transform(transform_data2)

df_train[transform_data1.columns] = std_transform_data1
df_test[transform_data2.columns] = std_transform_data2

df_train[x_list].head()

,thalach,oldpeak,slope,ca,sex,cp,exang,thal,age_gr
0,0.058612,0.206190,-2.158717,-0.659316,0.682656,0.887021,-0.693103,-0.527951,0.000000
1,1.170264,0.206190,-0.591572,-0.659316,0.682656,1.836950,-0.693103,1.132895,0.000000
2,1.897112,-0.902462,0.975574,0.434714,0.682656,-0.062909,-0.693103,1.132895,0.000000
3,0.058612,-0.717686,-0.591572,-0.659316,-1.464866,-1.012839,1.442786,-0.527951,-1.086134
4,0.999240,-0.902462,0.975574,-0.659316,0.682656,-0.062909,-0.693103,-0.527951,-2.172268


In [46]:
# 다중공선성 축소를 위한 PCA진행
from sklearn.decomposition import PCA
pca = PCA(n_components = 6) # 주성분을 몇개로 할지 결정
train_target = list(df_train['target'])
printcipalComponents1 = pca.fit_transform(df_train[x_list])
printcipalComponents2 = pca.fit_transform(df_test[x_list])
train = pd.DataFrame(data=printcipalComponents1, columns = [f'{i+1}'for i in range(6)])
train['target'] = train_target
test = pd.DataFrame(data=printcipalComponents2, columns = [f'{i+1}'for i in range(6)])
val_list = [f'{i+1}'for i in range(6)]
print(pca.explained_variance_ratio_) # 각각의 설명력을 표현
sum(pca.explained_variance_ratio_)

[0.30608205 0.14816272 0.11874532 0.10737341 0.09098099 0.08404527]


0.8553897452067849

In [47]:
# 로지스틱 회귀분석 테스트
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train[val_list], train.target)
print(classification_report(train.target, lr.predict(train[val_list])))

              precision    recall  f1-score   support

           0       0.90      0.81      0.85        68
           1       0.86      0.93      0.89        83

    accuracy                           0.87       151
   macro avg       0.88      0.87      0.87       151
weighted avg       0.88      0.87      0.87       151



In [48]:
# 정답파일 작성
predict = pd.read_csv(PATH + '\\sample_submission.csv')
predict.target = lr.predict(test[val_list])
predict.to_csv(PATH + '\\sample_submission(LR_PCA7).csv', index = False)
pd.read_csv(PATH + '\\sample_submission(LR_PCA7).csv').head()

,id,target
0,1,1
1,2,1
2,3,1
3,4,0
4,5,1


In [49]:
check1 = pd.read_csv(PATH + '\\sample_submission(LR_PCA7).csv')
check2 = pd.read_csv(PATH + '\\sample_submission(LR).csv')
for i in range(len(check1)):
    if check1.target[i] != check2.target[i]:
        print(i)

0
9
23
34
62
63
72
77
87
90
104
108
132
137


In [56]:
# 로지스틱 회귀분석 테스트
x_list = ['oldpeak', 'ca', 'cp', 'thal', 'sex', 'exang']
# 추가제거 : slope(oldpeak와 연관성이 높음)
# 추가제거 : thalach(oldpeak와 ca와의 연관성이 높음)
x_list = ['thalach', 'oldpeak', 'ca', 'sex', 'cp', 'thal']
# x_list = ['thalach', 'oldpeak', 'slope', 'ca', 'sex', 'cp', 'exang', 'thal', 'age_gr']
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(df_train[x_list], df_train.target)
print(classification_report(df_train.target, lr.predict(df_train[x_list])))

              precision    recall  f1-score   support

           0       0.91      0.87      0.89        68
           1       0.90      0.93      0.91        83

    accuracy                           0.90       151
   macro avg       0.90      0.90      0.90       151
weighted avg       0.90      0.90      0.90       151



In [57]:
# 정답파일 작성
predict = pd.read_csv(PATH + '\\sample_submission.csv')
predict.target = lr.predict(df_test[x_list])
predict.to_csv(PATH + '\\sample_submission(LR_CH5).csv', index = False)
pd.read_csv(PATH + '\\sample_submission(LR_CH5).csv').head()

,id,target
0,1,1
1,2,1
2,3,1
3,4,0
4,5,1


In [58]:
check1 = pd.read_csv(PATH + '\\sample_submission(LR_CH5).csv')
check2 = pd.read_csv(PATH + '\\sample_submission(LR).csv')
for i in range(len(check1)):
    if check1.target[i] != check2.target[i]:
        print(i)

0
84
118
129


In [53]:
# SVM 테스트
x_list = ['thalach', 'oldpeak', 'ca', 'sex', 'cp', 'thal']
# 추가제거 : slope(oldpeak와 중복해서 가중치가 들어갈 확률이 높음) / age_gr 제거
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm
svm = svm.SVC(kernel = "linear", C = 10)
svm.fit(df_train[x_list], df_train.target)
print(classification_report(df_train.target, svm.predict(df_train[x_list])))

              precision    recall  f1-score   support

           0       0.89      0.84      0.86        68
           1       0.87      0.92      0.89        83

    accuracy                           0.88       151
   macro avg       0.88      0.88      0.88       151
weighted avg       0.88      0.88      0.88       151



In [54]:
# 정답파일 작성
predict = pd.read_csv(PATH + '\\sample_submission.csv')
predict.target = svm.predict(df_test[x_list])
predict.to_csv(PATH + '\\sample_submission(SVM_CH2).csv', index = False)
pd.read_csv(PATH + '\\sample_submission(SVM_CH2).csv').head()

,id,target
0,1,1
1,2,1
2,3,1
3,4,0
4,5,1


In [55]:
check1 = pd.read_csv(PATH + '\\sample_submission(SVM_CH2).csv')
check2 = pd.read_csv(PATH + '\\sample_submission(LR).csv')
for i in range(len(check1)):
    if check1.target[i] != check2.target[i]:
        print(i)

0
11
25
84
99
114
118
129
